# SPAM FILTER

In [57]:
# Import modules

In [37]:
import os
import sys
import pandas as pd
import html2text as ht
import email
import bs4
from bs4 import BeautifulSoup
import re

# Functions 

## Read data using file crawler

In [ ]:
# Mails staan in folders, subfolders. FileCrawler loopt door de folders en subfolders heb en opent de mails.

In [2]:
# File crawler om data in te lezen
def FileCrawler(root, variable):
    for root, dirs, files in os.walk(root):
        for f in files: 
            with open(os.path.join(root,f)) as content:
                file_content = content.read()
                variable.append(file_content) 
    return

## Mails uit elkaar trekken

In [ ]:
# mails bestaan uit mail-informatie en het daadwerkelijke mailtje (dus from, to, etc.).
# SpamParser haalt de informatie los en geeft de "from, to, en body" als losse informatie.

In [87]:
def SpamParser(data):
    b = email.message_from_string(data)
    mail_from = b['from']
    mail_to = b['to']
    mail_body = []
    if b.is_multipart():
        for part in b.walk():
            if part.get_content_type() == 'text/plain':
                mail_body = part.get_payload()
            elif part.get_content_type() == 'text/html':
                mail_body = part.get_payload()
    else:
        mail_body = b.get_payload()
    return {'from':mail_from, 'to': mail_to,'body': mail_body}


## Line breaks verwijderen uit mail

In [127]:
def RemoveLineBreaks(mail):
    cleaned = re.sub("\n", " ", mail)
    cleaned = re.sub("\t", " ", cleaned)
    return cleaned

## Mails HTML to text

In [ ]:
# De spam mail bodies staan in html
# Clean_html: functie om html tags uit mailtje te halen. Werkt niet 100% (soms blijft er een enkele code staan) 
# maar wel beter dan html2text (encoding errors) en BeautifulSoup (die laat soms html staan zonder het te zeggen)

In [86]:
def clean_html(file):
    # First remove inline JavaScript/CSS:
    cleaned = re.sub(r"(?is)<(script|style).*?>.*?()", "", file)
    # Then remove html comments. 
    cleaned = re.sub(r"(?s)[\n]?", "", cleaned)
    # Next remove the remaining tags:
    cleaned = re.sub(r"(?s)<.*?>", " ", cleaned)
    # Finally deal with whitespace
    cleaned = cleaned.replace('&nbsp', ' ')
    cleaned = re.sub(r" ", " ", cleaned)
    cleaned = re.sub(r"^$", "", cleaned)
    cleaned = re.sub("''|,", "", cleaned)
    cleaned = re.sub(r"  ", " ", cleaned)

    return cleaned

# Clean and preprocess data

## Data inlezen

In [3]:
# HAM-data inlezen
root = "C:\Users\HOosterhuis\Documents\BDR\SPAMFILTER\HAM"
ham_data = []
FileCrawler(root, ham_data)

In [55]:
ham_data[2]

'Message-ID: <4154261.1075855758312.JavaMail.evans@thyme>\nDate: Thu, 2 Nov 2000 08:25:00 -0800 (PST)\nFrom: holly.heath@enron.com\nTo: robert.superty@enron.com, diane.cook@enron.com, heather.choate@enron.com, \n\tbob.hall@enron.com, vanessa.kyama@enron.com, bob.hall@enron.com, \n\tsheri.thomas@enron.com, lola.willis@enron.com, \n\tmary.solmonson@enron.com, marvia.jefferson@enron.com, \n\tjulissa.marron@enron.com, sally.beck@enron.com, \n\tpatti.thompson@enron.com\nSubject: Plan 2001 - Global Markets\nMime-Version: 1.0\nContent-Type: text/plain; charset=us-ascii\nContent-Transfer-Encoding: 7bit\nX-From: Holly Heath\nX-To: Robert Superty, Diane H Cook, Heather Choate, Bob M Hall, Vanessa Kyama, Bob M Hall, Sheri Thomas, Lola Willis, Mary Solmonson, Marvia Jefferson, Julissa Marron, Sally Beck, Patti Thompson\nX-cc: \nX-bcc: \nX-Origin: Beck-S\nX-FileName: sbeck.nsf\n\nAttached is a file containing all cost centers that are allocating to Global \nMarkets in the year 2001.  Could I please

In [5]:
# SPAM-data inlezen
root = "C:\Users\HOosterhuis\Documents\BDR\SPAMFILTER\SPAM"
spam_data = []
FileCrawler(root, spam_data)

In [53]:
spam_data[1]

'Return-Path: <Opal.Champion@e-a.com>\nDelivered-To: rait@bruce-guenter.dyndns.org\nReceived: (qmail 17070 invoked from network); 31 Jul 2004 22:20:13 -0000\nReceived: from localhost (localhost [127.0.0.1])\n  by bruce-guenter.dyndns.org ([192.168.1.3]); 31 Jul 2004 22:20:13 -0000\nReceived: from zak.futurequest.net ([127.0.0.1])\n  by localhost ([127.0.0.1])\n  with SMTP via TCP; 31 Jul 2004 22:20:13 -0000\nReceived: (qmail 10312 invoked from network); 31 Jul 2004 22:20:12 -0000\nReceived: from 200-148-23-125.dsl.telesp.net.br (unknown [200.148.23.125])\n  by zak.futurequest.net ([69.5.6.152])\n  with SMTP via TCP; 31 Jul 2004 22:20:10 -0000\nX-Message-Info: tfsp6LD1jly739XR167XFHBlNZ9urA0wVBmwLIU65D3\nReceived: (from eduardo@200.148.23.125)\n\tby aura4.3.190.168.0 (4.81.3/1.94.7) id u608GHGuP917022;\n\tSat, 31 Jul 2004 19:07:29 -0400\nMessage-ID: <247192680$5973SCEG$98@i-cable.com>\nDelivery-Date: Sun, 01 Aug 2004 00:10:29 +0100\nKeywords: econometric cavernous \nComments: align semi

## Ham-email omzetten

In [132]:
# Ham-mails: from, to en body extracten

In [133]:
ham_data_mail = [SpamParser(d) for d in ham_data]

In [134]:
ham_data_from = [x['from'] for  x in ham_data_mail]
ham_data_to = [x['to'] for  x in ham_data_mail]
ham_data_body = [x['body'] for  x in ham_data_mail]

In [135]:
ham_data_text = [RemoveLineBreaks(x) for x in ham_data_body]

In [136]:
ham_data_text[10]

'URGENT  The Associate and Analyst Recruiting Department will be conducting a number  of two hour workshops to review our recruiting and interview process for the  fall on-campus recruiting effort.  Critical information regarding our  on-campus interview process, revised evaluation forms and program structure  will be reviewed during these two hours sessions.  It is mandatory that all team members attend these workshops.  All team  members must attend in order to articulate and demonstrate the Enron  recruiting process.  Knowing how busy schedules are, we have made  arrangements to present these workshops in two hours sessions for a total of   40 workshops that will run during the last week of August, through the month  of September and end at mid October.  Listed below are the dates, location and times for each session.  Please  select a date and time and e-mail this information to my assistant, Dolores  Muzzy.  We can accommodate 25 participants at a time.  Dolores will take  dates a

In [145]:
len(ham_data_body)

19089

In [146]:
ham_data_body = filter(None, ham_data_body)

In [147]:
len(ham_data_body)

19089

## Spam email omzetten 

In [137]:
spam_data_mail = [SpamParser(d) for d in spam_data]

In [138]:
# Spam_data onderdelen los in lists opslaan, zodat verder gewerkt kan worden aan de (html-)body alleen

In [139]:
spam_data_from = [x['from'] for  x in spam_data_mail]
spam_data_to = [x['to'] for  x in spam_data_mail]
spam_data_body = [x['body'] for  x in spam_data_mail]

In [140]:
# Checken hoeveel mails zijn overgebleven

In [148]:
len(spam_data_body)

32848

In [149]:
# Lege mails er uit halen (zijn er maar een paar)

In [150]:
spam_data_body = filter(None, spam_data_body)

In [151]:
len(spam_data_body)

32848

## Spam data in HTML: omzetten naar ASCII

In [ ]:
# Mail body staat nog in HTML

In [152]:
spam_data_body[14]

'<html>\n<font color=\n\n#e5e6f6>Carolyn regretful watchfully Procrustes godly</font><br>\nSummer 2004 was too hot for the software manufacturers.<br>\nNo wonder! As the prices were reduced in 3-4 times.<br>\nThis was caused by the software glut on the world market.<br>\nOn the other hand the user who were not able or just had<br>\nno time to update their software now have the possibility<br>\nto do this almost free of charge.<br>\n<br>\n<b>\n<i>Read the whole article:</i><br>\n<a hrefrockerhref=http://ordinal.com href=\n\n"http://softbox24.info/index.php?s=0288">Year 2004. Sotware prices fall down.</a><br>\n<font size=-1>\xc2\xa9 Peter Lemelman</font>\n</font>\n</b>\n<font color=\n\n#e4e6e4>onerous reclaimers remunerate Lounsbury dictate</font><br>\n<font color=\n\n#fbfdf1>costed continued snooping digression Rhine</font><br>\n<font color=\n\n#f2f5f5>inseminate tilts instructs rejoice switchman</font><br>\n<font color=\n\n#e3e8e7>stomaching hurtling Brent gunners tortoises</font><br>\

In [153]:
# omzetten naar text met clean_html

In [154]:
spam_body_text = [clean_html(x) for x in spam_data_body]

In [155]:
spam_body_text[2]

'TOP quality software:  Special Offer #1:  Windows XP Professional+Microsoft Office XP Professional =3D only= $80 Special Offer #2:  Adobe - Photoshop 7 Premiere 7 Illustrator 10 =3D only $120 Special Offer #3:  Macromedia Dreamwaver MX 2004 + Flash MX 2004 =3D only $100 Also:    Windows 2003 Server Windows 2000 Workstation Windows 2000 Server      Windows 2000 Advanced Server   Windows 2000 Datacenter Windows NT 4.0 Windows Millenium Windows 98 Second Edition Windows 95 Office XP Professional  Office 2000  Office 97 MS Plus    MS SQL Server 2000 Enterprise Edition MS Visual Studio .NET Architect Edition  MS Encarta Encyclopedia Delux 2004 MS Project 2003 Professional MS Money 2004 MS Streets and Trips 2004 MS Works 7 MS Picture It Premium 9 MS Exchange 2003 Enterprise Server Adobe Photoshop Adobe PageMaker Adobe Illustrator           Adobe Acrobat 6 Professional Adobe Premiere Macromedia Dreamwaver MX 2004         Macromedia Flash MX 2004                  Macromedia Fireworks MX 2004 

# Create dataframe

# Divide data into training (60%), cross-validation (20%), and test set (20%) 

In [ ]:
train, validate, test = np.split(df.sample(frac=1), [int(.6*len(df)), int(.8*len(df))])


# Extract features

## most frequently occuring words in training set


# Train model

# Cross-validate model

## Evaluate performance
## Evaluate errors
## Extract new features?
## Choose model

# Test model

## Evaluate performance